In [27]:
import yfinance as yf 
from openai import OpenAI
import os 
import json 
import sys 
import requests
sys.path.append(r"C:\Users\o\agentflightbooking\fiancialAIAgent")

In [28]:
from openai import OpenAI
deepseek_client = OpenAI(
    api_key="sk-3da1e2817a2942ea8809277679bb05ec",
    base_url="https://api.deepseek.com/v1"
)

In [32]:
def reformuler_question_en_requete_usa(question, client):
    prompt = f"""
You are a legal research assistant for U.S. law.

Your job is to turn a user's natural language question into a focused Google search query
that finds relevant legal documents in PDF format on official U.S. government websites.

Instructions:
- Use only the key legal terms (state, topic, law name if known).
- End the query with: site:.gov filetype:pdf
- Be concise and relevant.
- give just one query 

Example:
Question: What is the law about privacy rights in California?
Query: california privacy law site:.gov filetype:pdf

---

User question:
\"{question}\"

Google search query:
"""

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return response.choices[0].message.content.strip()


In [33]:
question = 'What is the law of smooking weed in public in the u.s'
reforulation = reformuler_question_en_requete_usa(question=question, client=deepseek_client)

In [34]:
reforulation

'marijuana public consumption law U.S. site:.gov filetype:pdf'

In [35]:
import os
import requests

SERPAPI_API_KEY = os.getenv("SERA_API_KEY")

def chercher_pdfs_via_serpapi(query, api_key=SERPAPI_API_KEY):
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key,
        "num": 5, 
        "filetype": "pdf",
        "hl": "en"
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Erreur SerpAPI : {response.status_code} - {response.text}")

    data = response.json()
    results = data.get("organic_results", [])
    pdf_urls = []
    for result in results:
        link = result.get("link", "")
        if link.lower().endswith(".pdf"):
            pdf_urls.append(link)
    return pdf_urls


In [36]:
pdf_link = chercher_pdfs_via_serpapi(query=reforulation)

In [37]:
pdf_link

['https://ccb.nv.gov/wp-content/uploads/2022/08/Regulation-15_v072122.pdf',
 'http://publichealth.lacounty.gov/tob/docs/Clean%20Air%20Factsheets/Marijuana-in-Multi-Unit-Residential-Setting-2019-1.pdf',
 'https://health.hawaii.gov/medicalcannabis/files/2022/05/Cannabis-Policy-in-the-United-States-Implications-for-Public-Health.pdf',
 'https://www.congress.gov/crs_external_products/R/PDF/R44782/R44782.4.pdf',
 'https://www.congress.gov/crs_external_products/R/PDF/R44782/R44782.5.pdf']

In [38]:
import os
import requests
from datetime import datetime

def telecharger_pdf_dossier_temp(url_pdf, dossier_temp="data/temp"):
    os.makedirs(dossier_temp, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    nom_fichier = f"document_{timestamp}.pdf"
    chemin_complet = os.path.join(dossier_temp, nom_fichier)
    
    response = requests.get(url_pdf)
    if response.status_code != 200:
        raise Exception(f"Erreur téléchargement PDF : {response.status_code}")
    
    with open(chemin_complet, "wb") as f:
        f.write(response.content)
    
    return chemin_complet


In [39]:
save_pdf = telecharger_pdf_dossier_temp(url_pdf=pdf_link[0])

In [41]:
save_pdf

'data/temp\\document_20250719_211536_992815.pdf'

In [40]:
from pdf2image import convert_from_bytes
from typing import List, Optional
from PIL import Image
import os
import requests
import json
import pytesseract

def ocr(pdf):
    images = convert_from_bytes(pdf)
    ocr_texts = []
    
    for i, image in enumerate(images):
        image_path = f"page_{i + 1}.png"
        image.save(image_path, "PNG")
        print(f"Saved {image_path}")
    images_path = [f"page_{i + 1}.png" for i in range(len(images))]
    for image_path in images_path:
        image = Image.open(image_path)
        with Image.open(image_path) as image:
                if image.mode != 'RGB':
                    image = image.convert('RGB')
                
                text = pytesseract.image_to_string(
                    image, 
                    lang='eng',
                    config='--psm 6'  
                )   
                text = text.replace('\n', ' ').strip()
                ocr_texts.append(text)
        os.remove(image_path)
        print(f"Removed {image_path}")
        
    return ocr_texts

In [44]:
#read pdf 
with open(save_pdf, "rb") as f:
    pdf_bytes = f.read()

In [45]:
ocr_texts = ocr(pdf_bytes)

Saved page_1.png
Saved page_2.png
Saved page_3.png
Saved page_4.png
Saved page_5.png
Saved page_6.png
Saved page_7.png
Saved page_8.png
Saved page_9.png
Removed page_1.png
Removed page_2.png
Removed page_3.png
Removed page_4.png
Removed page_5.png
Removed page_6.png
Removed page_7.png
Removed page_8.png
Removed page_9.png


In [46]:
ocr_texts

['REGULATION 15 CANNABIS CONSUMPTION LOUNGE  15.010 Requirements for operation; posting of hours of operation.  15.015 Duties of cannabis establishment agent for a cannabis consumption lounge before sale to consumer.  15.020 Walid proof of identification of age of consumer required.  15.025 Prohibition on sale that exceeds maximum usable quantity of cannabis.  15.030 Products required to be offered for sale; restrictions on sale of other products; restrictions on advertising.  15.035 Storage and location of products; disclosure of cannabis independent testing laboratory performing quality assurance tests upon request of consumer; approved soutces of products for sale; maintenance and availability of certificate of analysis.  15.040 On-site requirements to operate a cannabis consumption lounge.  15.045 Employee Training and Consumer Education.  15.050 Separate room for cannabis smoking, vaping, and inhalation.  15.055 Ventilation of the cannabis consumption lounge.  15.060 Notices and a

In [47]:
def ai_agent_analyze_ocr(ocr_texts, question,client):

    full_text = "\n\n".join(ocr_texts)
    
    prompt = f"""
You are a legal assistant helping to analyze PDF documents extracted by OCR.

User question: {question}

Below is the extracted text from the PDF document (from OCR):

{full_text}

Please provide a clear and concise answer to the user's question based on the content of the document.
"""
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful legal assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=800
    )
    
    answer = response.choices[0].message.content.strip()
    return answer

In [48]:
final_response = ai_agent_analyze_ocr(ocr_texts,question,client=deepseek_client)

In [50]:
print(final_response)

The extracted text from the PDF outlines regulations for **cannabis consumption lounges** in Nevada, but it does not directly address the general **law on smoking weed in public in the U.S.**  

### Key Takeaways on Public Consumption:  
1. **Federal Law**: Cannabis remains illegal under federal law, though enforcement varies.  
2. **State Laws**: Most U.S. states where cannabis is legal (medically or recreationally) **prohibit public consumption**, including smoking in parks, streets, or other public spaces.  
3. **Nevada-Specific Rules (from the document)**:  
   - Cannabis consumption lounges must ensure consumption is **not visible to the public** (Regulation 15.090).  
   - Outdoor consumption areas must be **obscured from public view** (e.g., by walls or barriers).  
   - Removing cannabis from the lounge is **illegal** (15.030).  

### General U.S. Rule:  
Smoking weed in public is **typically prohibited** in legal states and can result in fines or penalties. Consumption is usua